Read in raw netCDF inflow plane data from AMR-Wind sampling files, and reformat the data so it is dimensioned properly.

In [1]:
from datetime import datetime
import numpy as np
import matplotlib.pyplot as plt
from netCDF4 import Dataset
import xarray as xr
from matplotlib.colors import TwoSlopeNorm
from pathlib import Path
import pandas as pd
import h5py

import dask.array as da

In [2]:
### Read in data and do some minimal processing
parent_dir = Path('/scratch/orybchuk/wakedynamics/bcs-ldm/simulations/072415/large_campaigns/precursor/post_processing')
outdir = Path('/scratch/orybchuk/wakedynamics/bcs-ldm/data/072415/post_processing/nc_dimensioned')
outdir.mkdir(exist_ok=True)

campaign_files = list(parent_dir.glob('sampling*.nc'))
campaign_files.sort()
n_campaigns = len(campaign_files)
print(f"Number of campaigns: {n_campaigns}")

Number of campaigns: 345


# Common parameters

In [3]:
### Set up parameters common to all campaigns
## User inputs
varlist = ['velocityx', 'velocityy', 'velocityz', 'temperature', 'tke']  # Manually build this
use_dask = False
frequency_output = 1

## Data from simulations
with h5py.File(campaign_files[0]) as f_h5py:
    # Deal with time
    time = f_h5py['time'][:]
    time = np.round(time, 4)  # Round to deal with weird float behavior
    timestep = time[1] - time[0]
    
    # List of variables to process
    tmp_meas = list(f_h5py)[0]
    full_varlist = list(f_h5py[tmp_meas].keys())
for var in varlist:
    assert var in full_varlist
print("varlist:", varlist)

varlist: ['velocityx', 'velocityy', 'velocityz', 'temperature', 'tke']


# Reformat inflow BCs at $x=-3D$

In [4]:
### Reformat inflow boundary conditions
print("Start reformatting data", datetime.now())

curr_varname = 'inflow-bc-3D'

for fcampaign in campaign_files:
    campaign_id = fcampaign.name[8:12]
    
    if int(campaign_id) % 10 == 0: print(campaign_id+' - '+str(datetime.now()))
    
    ## Metadata for inflow BCs
    with h5py.File(fcampaign) as f_h5py:
        f_bc = f_h5py[curr_varname]
        bc_coords_all = f_bc['coordinates'][:]
        time_file = f_h5py['time'][:]
        bc_xcoords = np.sort(np.unique(bc_coords_all[:,0]))
        bc_ycoords = np.sort(np.unique(bc_coords_all[:,1]))
        bc_zcoords = np.sort(np.unique(bc_coords_all[:,2]))
    bc_coords = {'time':time_file, 'x': bc_xcoords, 'y': bc_ycoords, 'z': bc_zcoords}
    ds_bc = xr.Dataset(bc_coords)
    
    for var in varlist:
        if use_dask:
            raise NotImplementedError

            # TODO: check the below code
            curr_da = da.from_array(f_xy[var], chunks=chunksize_xy)
            da_reshaped = curr_da.reshape((curr_da.shape[0], len(xy_zcoords), len(xy_ycoords), len(xy_xcoords)))
            ds_xy[var] = (('time', 'z', 'y', 'x'), da_reshaped)
        else:
            with h5py.File(fcampaign) as f_h5py:
                curr_arr = f_h5py[curr_varname][var][:]
            arr_reshaped = curr_arr.reshape((len(time_file), len(bc_zcoords), len(bc_ycoords), len(bc_xcoords)))
            arr_transposed = np.transpose(arr_reshaped, axes=[0,3,2,1])
            ds_bc[var] = (('time', 'x', 'y', 'z'), arr_transposed)
    ds_bc.to_netcdf(Path(outdir,f"{curr_varname}{campaign_id}.nc"))
    
print("End reformatting data", datetime.now())

Start reformatting data 2024-02-12 21:25:48.371281
0000 - 2024-02-12 21:25:48.371642
0010 - 2024-02-12 21:47:33.141924
0020 - 2024-02-12 22:17:16.898017
0030 - 2024-02-12 22:48:24.161831
0040 - 2024-02-12 23:18:14.888077
0050 - 2024-02-12 23:49:34.969857
0060 - 2024-02-13 00:18:03.752079
0070 - 2024-02-13 00:49:45.980047
0080 - 2024-02-13 01:18:40.770895
0090 - 2024-02-13 01:49:45.178376
0100 - 2024-02-13 02:19:59.192429
0110 - 2024-02-13 02:52:24.369191
End reformatting data 2024-02-13 03:18:56.121269
